Table of Contents: 
1. Importing Libraries and Dataframes
2. Wrangling the data so that it follows consistent logic
3. Completing the fundamental data quality and consistency checks
4. Combining the customer data with the rest of prepared Instacart data


### 1. Importing Libraries and Dataframes

In [1]:
# Importing Analysis Libraries
import pandas as pd
import numpy as np
import os

In [3]:
# Importing Dataframes
path = r'C:\Users\franz\Documents\Exercise_4'
df = pd.read_csv(os.path.join(path, '02_Data', 'original_data', 'customers_4.9.csv'), usecols = all)
ords_prods_merge_old = pd.read_pickle(os.path.join(path, '02_Data', 'prepared_data', 'orders_products_incl_new_variables_4.9_dropped.pkl'))

### 2. Wrangling the data so that it follows consistent logic

for example, rename columns with illogical names and drop columns that don’t add anything to your analysis.

In [3]:
df.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [7]:
# renaming columns with illogical names
df.rename(columns = {'First Name':'first_name', 'Surnam':'last_name', 'Gender':'gender', 'State':'state', 'Age':'age', 'STATE':'state', 'fam_status':'marital_status'}, inplace = True)

#### drop columns that don´t add anything to your analysis: I did not find any irrelevant column

In [8]:
df.head()

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


### 3. Completing the fundamental data quality and consistency checks

for example, check for and address missing values and duplicates, and convert any mixed-type data.

#### Mixed data types:

In [6]:
# Checking for mixed-type columns

for col in df.columns.tolist():
  weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[weird]) > 0:
    print (col)

first_name


In [9]:
# Fixing the mixed-type column "first_name" and assigning the datatype string, as names are always strings

df['first_name'] = df['first_name'].astype('str')

In [10]:
# Checking for mixed-type columns again

for col in df.columns.tolist():
  weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[weird]) > 0:
    print (col)

In [9]:
df.dtypes

user_id            int64
first_name        object
last_name         object
gender            object
state             object
age                int64
date_joined       object
n_dependants       int64
marital_status    object
income             int64
dtype: object

#### Now, there are no mixed data types anymore.

In [10]:
# Find missing values:
df.isnull().sum()

user_id           0
first_name        0
last_name         0
gender            0
state             0
age               0
date_joined       0
n_dependants      0
marital_status    0
income            0
dtype: int64

#### We do not have any missing values

In [11]:
# looking for full duplicates within the dataframe:
df_dups = df[df.duplicated()]

In [12]:
df_dups.head()

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,marital_status,income


#### We do not have any duplicate rows

In [11]:
# data export
df.to_pickle(os.path.join(path, '02_data','prepared_data', 'customers.pkl'))

### 4. Combining the customer data with the rest of prepared Instacart data

#### Find the common column(s):

In [13]:
ords_prods_merge_old.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices',
       'order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'add_to_cart_order',
       'reordered', 'price_range_loc', 'busiest_day', 'busiest_days',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'max_price',
       'min_price', 'mean_price', 'spending_flag',
       'median_days_since_prior_order', 'order_frequency_flag'],
      dtype='object')

In [14]:
df.columns

Index(['user_id', 'first_name', 'last_name', 'gender', 'state', 'age',
       'date_joined', 'n_dependants', 'marital_status', 'income'],
      dtype='object')

#### The key column is user_id

#### Performing some pre-checks:

In [15]:
df.dtypes

user_id            int64
first_name        object
last_name         object
gender            object
state             object
age                int64
date_joined       object
n_dependants       int64
marital_status    object
income             int64
dtype: object

In [16]:
ords_prods_merge_old.dtypes

product_id                         int32
product_name                      object
aisle_id                            int8
department_id                       int8
prices                           float16
order_id                           int32
user_id                            int32
order_number                       int32
orders_day_of_week                 int32
order_hour_of_day                  int32
days_since_prior_order           float64
add_to_cart_order                  int16
reordered                           int8
price_range_loc                   object
busiest_day                       object
busiest_days                      object
busiest_period_of_day             object
max_order                          int32
loyalty_flag                      object
max_price                        float16
min_price                        float16
mean_price                       float16
spending_flag                     object
median_days_since_prior_order    float64
order_frequency_

In [17]:
#assigning int32 type to the user_id column in df (as it is the case in ords_prods_merge_old)
#change data types to free some memory for the merge
df['user_id'] =df['user_id'].astype('int32')
df['age'] =df['age'].astype('int32')
df['income'] =df['income'].astype('int32')

In [18]:
df.head()

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [19]:
df.dtypes

user_id            int32
first_name        object
last_name         object
gender            object
state             object
age                int32
date_joined       object
n_dependants       int64
marital_status    object
income             int32
dtype: object

In [20]:
df.shape

(206209, 10)

In [21]:
ords_prods_merge_old.shape

(32404859, 25)

Deleting some columns in ords_prods_merge_old to save memory

In [22]:
ords_prods_merge_old = ords_prods_merge_old.drop(columns = ['_merge'])

In [23]:
ords_prods_merge_old = ords_prods_merge_old.drop(columns = ['Unnamed: 0_x'])

In [24]:
ords_prods_merge_old = ords_prods_merge_old.drop(columns = ['Unnamed: 0_y'])

In [25]:
ords_prods_merge_old = ords_prods_merge_old.drop(columns = ['exists'])

In [26]:
ords_prods_merge_old['add_to_cart_order'] =ords_prods_merge_old['add_to_cart_order'].astype('int16')
ords_prods_merge_old['days_since_prior_order'] =ords_prods_merge_old['days_since_prior_order'].astype('float16')
ords_prods_merge_old['median_days_since_prior_order'] =ords_prods_merge_old['median_days_since_prior_order'].astype('float16')

In [27]:
# Data Export: 
# ords_prods_merge_old.to_pickle(os.path.join(path, '02_data','prepared_data', 'orders_products_incl_new_variables_4.9_dropped.pkl'))

### Performing the actual merge:

In [28]:
ords_prods_merge_old.info()

In [29]:
# direct merge does not work due to memory errors, thus it will not be applied
ords_prods_cust_merge = df.merge(ords_prods_merge_old, on = 'user_id', indicator = 'exists_4.9')